# Estimering av diamantpriser
En XGB modell för att estimera priset på en diamant

Först kommer alltid importer

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'